In [ ]:
def get_predictions(val_loader, model, threshold=0.5):
    model.eval()  # Set the model to evaluation mode
    predictions = []

    with torch.no_grad():  # Disable gradient calculation
        for input_ids, attention_mask, extra_features in val_loader:  # No labels
            # Move inputs to the same device as the model (e.g., GPU if available)
            input_ids = input_ids.to(model.fc[0].weight.device)
            attention_mask = attention_mask.to(model.fc[0].weight.device)
            extra_features = extra_features.to(model.fc[0].weight.device)

            # Get model outputs
            logits = model(input_ids, attention_mask, extra_features)
            probabilities = torch.sigmoid(logits)  # Convert logits to probabilities
            predictions.append((probabilities > threshold).cpu().numpy())  # Apply threshold and store predictions

    # Concatenate all batches into a single numpy array
    predictions = np.concatenate(predictions, axis=0)
    return predictions
    # sig = nn.Sigmoid() 
    # yhat = sig(model(X_val)).detach().numpy()
    # y_pred = yhat > threshold
    
    # return y_pred

# y_pred = get_predictions(torch.Tensor(final_val_features), model, 0.45)
# # print(y_pred)

# # Create a DataFrame to save to CSV
# val_data_with_pred = pd.DataFrame(y_pred, columns=['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])  # Adjust column names as per your features
# # val_data_with_pred['True_Label'] = y_test
# # val_data_with_pred['Predictions'] = dummy_predictions

# val_data_with_pred = val_data_with_pred.astype(int)

# val_data_with_pred['id'] = val['id']

# val_data_with_pred = val_data_with_pred[['id', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']]

# # Save to CSV
# current_time = datetime.datetime.now()
# formatted_time = current_time.strftime('%Y-%m-%d_%H_%M_%S')

# val_data_with_pred.to_csv(f'../results/pred_eng_a_{formatted_time}.csv', index=False)

# print(val_data_with_pred)

In [ ]:
# DO 10 different prediction files with epochs 100, 200, 300, 400, 500, 600, 700, 800, 900 and 1000 Model parameters

# Get 10 differen prediction files

bert_val_text = tokenizer(
    val["text"].tolist(),
    return_tensors="pt",
    add_special_tokens=True,
    padding="max_length",
    truncation=True,
    max_length=128
)

val_input_ids = bert_val_text["input_ids"]
val_attention_mask = bert_val_text["attention_mask"]

# Convert val_lr_features to PyTorch tensor
val_lr_features_tensor = torch.tensor(val_lr_features, dtype=torch.float32)

# Create TensorDataset for validation data (without labels)
val_dataset = TensorDataset(val_input_ids, val_attention_mask, val_lr_features_tensor)

# Validation DataLoader
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# for input_ids, attention_mask, extra_features, _ in val_loader:  # Assuming DataLoader includes labels
#     # Move inputs to the same device as the model
#     input_ids = input_ids.to(model.fc[0].weight.device)
#     attention_mask = attention_mask.to(model.fc[0].weight.device)
#     extra_features = extra_features.to(model.fc[0].weight.device)

#     # Get model outputs
#     logits = model(input_ids, attention_mask, extra_features)
#     probabilities = torch.sigmoid(logits)  # Convert logits to probabilities
#     predictions.append((probabilities > threshold).cpu().numpy())  # Apply threshold


for i in range(4):
    epoch = i*100
    model.load_state_dict(torch.load(f'./24-12-24/net_epoch_{epoch}.pth', weights_only=True))
    y_pred = get_predictions(val_loader, model, 0.45)
    # print(y_pred)

    # Create a DataFrame to save to CSV
    val_data_with_pred = pd.DataFrame(y_pred, columns=['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])  # Adjust column names as per your features
    # val_data_with_pred['True_Label'] = y_test
    # val_data_with_pred['Predictions'] = dummy_predictions

    val_data_with_pred = val_data_with_pred.astype(int)

    val_data_with_pred['id'] = val['id']

    val_data_with_pred = val_data_with_pred[['id', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']]

    # Save to CSV
    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime('%Y-%m-%d_%H_%M_%S')

    val_data_with_pred.to_csv(f'../results/alt5/alt5_epoch_{epoch}_pred_eng_a_{formatted_time}.csv', index=False)

    print(val_data_with_pred)